In [2]:
import polars as pl
import altair as alt


In [3]:
benchmark_df = pl.read_csv("../bench-feather.csv")

all_correct = benchmark_df.select(pl.col("correct").all()).item(0,0)

if not all_correct:
    print("ALARM !!!")

benchmark_df

benchmark,correct,environment,execution_time_us,init_runtime_us,iteration,load_program_us
str,bool,str,i64,i64,i64,i64
"""tarfind""",true,"""jerryscript""",7893220,99,0,13785
"""tarfind""",true,"""jerryscript""",7891387,98,1,13785
"""tarfind""",true,"""jerryscript""",7891387,98,2,13784
"""tarfind""",true,"""jerryscript""",7891387,99,3,13785
"""tarfind""",true,"""jerryscript""",7891387,99,4,13785
…,…,…,…,…,…,…
"""tarfind""",true,"""wamr""",1014189,452,0,2152
"""tarfind""",true,"""wamr""",1012546,226,1,2122
"""tarfind""",true,"""wamr""",1009306,227,2,2121


In [6]:
benchmark_avg_over_iteration = benchmark_df.group_by("environment").agg(
    pl.col("execution_time_us").mean(), (pl.col("init_runtime_us").mean() + pl.col("load_program_us").mean()).alias("load_time_us")
    )

execution_time = alt.Chart(benchmark_avg_over_iteration).encode(
    alt.X("environment", title="execution environment").sort("y"),
    alt.Y("execution_time_us", title="execution time in µs" ),
).mark_bar()


load_time = alt.Chart(benchmark_avg_over_iteration).encode(
    alt.X("environment", title="execution environment").sort("y"),
    alt.Y("load_time_us", title="load time in µs" ),
).mark_bar()

execution_time.display(scaleFactor=2)
load_time.display(scaleFactor=2)

alt.Chart(...)

alt.Chart(...)

In [7]:
native_execution_time = benchmark_avg_over_iteration.filter(pl.col("environment").eq("native")).select("execution_time_us").item(0,0)

# print("native_execution_time", native_execution_time)

benchmark_avg_over_iteration.with_columns(
    pl.col("execution_time_us").truediv(native_execution_time).alias("times native")
)

environment,execution_time_us,load_time_us,times native
str,f64,f64,f64
"""jerryscript""",7891753.6,13883.4,327.141017
"""native""",24123.4,0.0,1.0
"""wamr""",1.011616e6,2408.2,41.935051
"""micropython""",5619891.6,42535.4,232.964325
